# Evaluation of dynamic simulated model

In [1]:
import pandas as pd
import numpy as np
import os
import pickle

from sklearn.metrics import make_scorer
from scipy import optimize
from scipy import integrate

import matplotlib as mpl
import matplotlib.pyplot as plt


# SIR dynamic model
def SIR_model_t(SIR,t,beta,gamma):

    ''' Simple SIR model
        S: susceptible population
        t: time step, mandatory for integral.odeint
        I: infected people
        R: recovered people
        beta: 
        
        overall condition is that the sum of changes (differnces) sum up to 0
        dS+dI+dR=0
        S+I+R= N (constant size of population)
    
    '''
    
    S,I,R=SIR
    dS_dt=-beta*S*I/N0          #S*I is the 
    dI_dt=beta*S*I/N0-gamma*I
    dR_dt=gamma*I
    return dS_dt,dI_dt,dR_dt

def SIR_model(SIR,beta,gamma):
    ''' Simple SIR model
        S: susceptible population
        t: time step, mandatory for integral.odeint
        I: infected people
        R: recovered people
        beta: 
        
        overall condition is that the sum of changes (differnces) sum up to 0
        dS+dI+dR=0
        S+I+R= N (constant size of population)
    
    '''
    
    S,I,R=SIR
    dS_dt=-beta*S*I/N0          
    dI_dt=beta*S*I/N0-gamma*I
    dR_dt=gamma*I
    return dS_dt,dI_dt,dR_dt

def fit_odeint(x, beta, gamma):
    '''
    helper function for the integration
    '''
    return integrate.odeint(SIR_model_t, (S0, I0, R0), x, args=(beta, gamma))[:,1] # we only would like to get dI


def set_parameters(df_analyse, country):
    '''
    initalize parameters for 
    '''
 
    # initlaize population for particular country
    population = {
        'Brazil' : 212000000,
        'Spain' : 46000000,
        'Germany' : 83000000
    }
    
    # get index for country with more than 1000 cases as start point
    n = df_analyse[df_analyse[country] >= 1000][country] .idxmin()
#     print(n)
    
    # store infected cases 
    ydata = np.array(df_analyse[country][n:])
    
    # get day index
    t=np.arange(0, len(ydata))
    
  
    return ydata, t, population[country]

def SIR_dynamic_model(country, interval = 7):
    
    global S0, I0, R0, N0, t
     # load processed data
    df_analyse=pd.read_csv('../data/processed/COVID_small_flat_table.csv',sep=';') 
    
    ## set parameters ##
    ydata, t, population_size = set_parameters(df_analyse, country)
       
    # initalization for SIR_model
    N0= population_size  # population
    I0=ydata[0]    # infected 
    S0=N0-I0       # suspected
    R0=0           # recovered
    
    #initaliye hzperparameters
    beta=0.4
    gamma=0.4
    
    ######## Among three solutions, interval fit is selected ##########
    
    # initalize array
    interval_fitted = np.array([])

    # initalize array of SIR values
    SIR=np.array([S0,I0,R0])
    
    for i in range(len(ydata)):
        
        # select interval data
        interval_data = ydata[i*interval:(i*interval)+interval]
        interval_t = np.arange(len(interval_data))
        
        # check for condition
        if interval_data.size == 0:
            break

        #Re-initialize SIR for each interval
        I0 = interval_data[0]                  
        S0 = N0-I0 
        R0 = SIR[2] 
    
        # optimize curvefit
        popt, pcov = optimize.curve_fit(fit_odeint, interval_t, interval_data, maxfev=1500)
    
        # Recalculate SIR with new_delta
        new_delta = SIR_model(SIR,*popt)
        SIR = SIR + new_delta
        
        # temporary fit for interval
        temp_fit = fit_odeint(interval_t,*popt)
        
        # fit with other interval data
        interval_fitted = np.hstack((interval_fitted, temp_fit))
        
    return ydata, interval_fitted

if __name__ == '__main__':
    

    Brazil_ydata, Brazil_fitted = SIR_dynamic_model('Brazil')
    Germany_ydata, Germany_fitted = SIR_dynamic_model('Germany')
    Spain_ydata, Spain_fitted = SIR_dynamic_model('Spain')
    cocn_dict = {'Brazil_ydata' : Brazil_ydata, 'Brazil_fitted' : Brazil_fitted }
    df_Brazil= pd.DataFrame(cocn_dict)
    
    cocn_dict = {'Germany_ydata' : Germany_ydata, 'Germany_fitted' : Germany_fitted }
    df_germany = pd.DataFrame(cocn_dict)
    
    cocn_dict = {'Spain_ydata': Spain_ydata, 'Spain_fitted' : Spain_fitted}
    df_SA = pd.DataFrame(cocn_dict)
    
    dynamic_model = pd.concat([df_Brazil, df_SA, df_germany], axis=1)

    dynamic_model.to_csv('../data/processed/COVID_infected_cases_dynamic_model.csv', sep = ';', index=False)

In [14]:
dynamic_model.head()

,Brazil_ydata,Brazil_fitted,Spain_ydata,Spain_fitted,Germany_ydata,Germany_fitted
0,1021.0,1021.000000,1073.0,1073.000000,1040,1040.000000
1,1546.0,1356.678388,1695.0,1509.848120,1176,1408.468260
2,1924.0,1759.046443,2277.0,2124.434932,1457,1865.217117
3,2247.0,2209.527043,2277.0,2988.964304,1908,2397.941431
4,2554.0,2667.222441,5232.0,4204.862191,2078,2967.681339


# Visualization Dash board

In [ ]:
import dash
dash.__version__
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output,State

import plotly.graph_objects as go
import plotly.express as px


# Load processed file 
df_plot=pd.read_csv('../data/processed/COVID_infected_cases_dynamic_model.csv', sep=';')  


fig = go.Figure()


app = dash.Dash(__name__)

app.layout = html.Div([

    dcc.Markdown('''
    #  Applied Data Science on COVID-19 data (Dashboard-2)

    In the dashboard, the SIR model is used for calculation for dynamic simulated infection spread. 
    The simulated values of curve are obtained after finding optimized values of infection rate and recovery rate as interval fit to data for every 7 days.

    '''),

    dcc.Markdown('''
    ## Select a country for visualization
    '''),


    dcc.Dropdown(
        id='country_drop_down',
        options=[ {'label': each,'value':each} for each in ['Germany', 'Spain', 'Brazil']],
        value='Spain',  #  pre-selected
        multi=False
    ),


    dcc.Graph(id='dynamic', figure= fig, )
])

@app.callback(
    Output('dynamic', 'figure'),
    [Input('country_drop_down', 'value')])

def update_figure(country):
    
    # column names
    ydata = '{}_ydata'.format(country)
    fitted = '{}_fitted'.format(country)
    
    # select data
    df_select = df_plot[[ydata, fitted]]
    
    
    
    # Add traces
    traces=[]
    traces.append(go.Scatter(x=df_select.index, y=df_select[ydata],
                        mode='lines+markers', name='{}'.format(country),
                             marker=dict(
                         color='rgb(125, 178, 102)',
                         size=10,
                         line=dict(
                            color='DarkSlateGrey',
                            width=1
                          ),

                             ))),
                 
                 
    traces.append(go.Bar( x = df_select.index, y=df_select[fitted],
                        name='{}_simulated'.format(country)
                 ) )
             
    
    return {
            'data': traces,
             
            'layout': dict (
                width = 1280,
                height = 720,
                
                xaxis={'title':'No. of days of infection',
                        'tickangle':-45,
                        'nticks':20,
                        'tickfont':dict(size=14,color="#171717"),
                      },

                yaxis= {'title':'No. of infected cases (non-log scale)',
                        'tickangle':-45,
                        'nticks':20,
                        'tickfont':dict(size=14,color="#171717"),
                      },
                xaxis_rangeslider_visible = True
        )
    }


# dashboard
if __name__ == '__main__':

    app.run_server(debug=True, use_reloader=False, port = 8853)

Dash is running on http://127.0.0.1:8853/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
